In [2]:
! pip install matplotlib


You should consider upgrading via the 'C:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
! pip install pytorch-crf



In [5]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

from torchcrf import CRF

def load_glove_embeddings(file_path, word_to_index, embedding_dim):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    num_words = len(word_to_index) + 1  # Add 1 for the padding token
    embedding_matrix = np.zeros((num_words, embedding_dim))

    for word, index in word_to_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
        else:
            embedding_matrix[index] = embeddings_index.get("<unk>", np.zeros(embedding_dim))

    return embedding_matrix


with open("processed_train_data.json", "r") as f_train, open("processed_val_data.json", "r") as f_val:
    processed_train_data = json.load(f_train)
    processed_val_data = json.load(f_val)


texts_train = [entry["text"] for entry in processed_train_data.values()]
labels_train = [entry["labels"] for entry in processed_train_data.values()]

texts_val = [entry["text"] for entry in processed_val_data.values()]
labels_val = [entry["labels"] for entry in processed_val_data.values()]


word_to_index = {}
index = 1 
for entry in processed_train_data.values():
    tokens = entry["text"].split() 
    for token in tokens:
        if token not in word_to_index:
            word_to_index[token] = index
            index += 1

word_to_index["<unk>"] = 0
embedding_dim = 100

glove_embeddings_path = "glove.6B.100d.txt"  
embedding_matrix = load_glove_embeddings(glove_embeddings_path, word_to_index, embedding_dim)

label_to_index = {'O': 0, 'B': 1, 'I': 2}

X_train = [[word_to_index.get(token, word_to_index["<unk>"]) for token in text.split()] for text in texts_train]
y_train = [[label_to_index[label] for label in entry] for entry in labels_train] 
X_val = [[word_to_index.get(token, word_to_index["<unk>"]) for token in text.split()] for text in texts_val]
y_val = [[label_to_index[label] for label in entry] for entry in labels_val] 



class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.LongTensor(self.X[idx]), torch.LongTensor(self.y[idx])


def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs, targets


train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


class BiLSTMCRF(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size):
        super(BiLSTMCRF, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), padding_idx=0)
        self.bilstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, 
                              batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.crf = CRF(output_size, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.bilstm(x)
        logits = self.fc(lstm_out)
        return logits
    def calculate_crf_loss(self, emissions, tags):
        return -self.crf(emissions, tags)

hidden_size = 100
output_size = len(label_to_index)
model = BiLSTMCRF(embedding_matrix, hidden_size, output_size)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 60

train_losses = []
val_losses = []
train_f1_scores = []
val_f1_scores = []


for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        emissions = model(inputs)
        crf_loss = model.calculate_crf_loss(emissions, targets)
        crf_loss.backward()
        optimizer.step()

    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, output_size), targets.view(-1))
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=2).cpu().numpy()
            targets = targets.cpu().numpy()

            all_preds.extend(preds)
            all_targets.extend(targets)

        val_loss /= len(val_loader)
        val_macro_f1 = f1_score(np.concatenate(all_targets, axis=0), np.concatenate(all_preds, axis=0), average='macro')
        train_losses.append(loss.item())
        val_losses.append(val_loss)
        train_f1_scores.append(val_macro_f1) 
        val_f1_scores.append(val_macro_f1)  

        if epoch == num_epochs - 1:
            print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}')
            print(f'Validation Macro F1: {val_macro_f1:.4f}')

# Plotting Loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.savefig('plots/Bilstm_glove/Loss.png') 
plt.show()

# Plotting F1 Score
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_f1_scores, label='Training Macro-F1')
plt.plot(range(1, num_epochs+1), val_f1_scores, label='Validation Macro-F1')
plt.xlabel('Epochs')
plt.ylabel('Macro-F1 Score')
plt.title('Training and Validation Macro-F1 Score Over Epochs')
plt.legend()
plt.savefig('plots/Bilstm_glove/Macro.png') 
plt.show()
torch.save(model.state_dict(),"models/BiLSTMCRF_glove.pth")

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

Testing 

In [13]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

with open("processed_test_data.json", "r") as f_test:
    processed_test_data = json.load(f_test)

texts_test = [entry["text"] for entry in processed_test_data.values()]
labels_test = [entry["labels"] for entry in processed_test_data.values()]

X_test = [[word_to_index.get(token, word_to_index["<unk>"]) for token in text.split()] for text in texts_test]
y_test = [[label_to_index[label] for label in entry] for entry in labels_test]

test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Ensure that the model is in evaluation mode
model.eval()

all_preds_test = []
all_targets_test = []

with torch.no_grad():
    for inputs_test, targets_test in test_loader:
        outputs_test = model(inputs_test)
        preds_test = torch.argmax(outputs_test, dim=2).cpu().numpy()
        targets_test = targets_test.cpu().numpy()

        all_preds_test.extend(preds_test)
        all_targets_test.extend(targets_test)

# Concatenate results
all_preds_test = np.concatenate(all_preds_test, axis=0)
all_targets_test = np.concatenate(all_targets_test, axis=0)

# Calculate accuracy and F1 score
test_accuracy = accuracy_score(all_targets_test, all_preds_test)
test_macro_f1 = f1_score(all_targets_test, all_preds_test, average='macro')

print(f'Final Test Accuracy: {test_accuracy:.4f}, Final Test Macro F1: {test_macro_f1:.4f}')


Final Test Accuracy: 0.9681, Final Test Macro F1: 0.6858
